In [131]:
import numpy as np
import pandas as pd
import cudf
import os
from cuml import Ridge as cuRidge
from sklearn.linear_model import Ridge as skRidge
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error

In [132]:
!nvidia-smi

Thu Jul 11 14:34:28 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.104      Driver Version: 410.104      CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:06:00.0 Off |                    0 |
| N/A   37C    P0    56W / 300W |   1714MiB / 16130MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [133]:
trainc = cudf.read_csv('train.csv')
testc = cudf.read_csv('test.csv')

In [134]:
print(trainc.head())

   Id  Elevation  Aspect  Slope  Horizontal_Distance_To_Hydrology  Vertical_Distance_To_Hydrology  Horizontal_Distance_To_Roadways ...  Cover_Type
0   1       2596      51      3                               258                               0                              510 ...           5
1   2       2590      56      2                               212                              -6                              390 ...           5
2   3       2804     139      9                               268                              65                             3180 ...           2
3   4       2785     155     18                               242                             118                             3090 ...           2
4   5       2595      45      2                               153                              -1                              391 ...           5
[48 more columns]


In [135]:
import cudf

In [136]:
train =cudf.read_csv('train.csv')
test =cudf.read_csv('test.csv')

In [137]:
print(train.head())

   Id  Elevation  Aspect  Slope  Horizontal_Distance_To_Hydrology  Vertical_Distance_To_Hydrology  Horizontal_Distance_To_Roadways ...  Cover_Type
0   1       2596      51      3                               258                               0                              510 ...           5
1   2       2590      56      2                               212                              -6                              390 ...           5
2   3       2804     139      9                               268                              65                             3180 ...           2
3   4       2785     155     18                               242                             118                             3090 ...           2
4   5       2595      45      2                               153                              -1                              391 ...           5
[48 more columns]


In [138]:
print(test.head())

      Id  Elevation  Aspect  Slope  Horizontal_Distance_To_Hydrology  Vertical_Distance_To_Hydrology  Horizontal_Distance_To_Roadways ...  Soil_Type40
0  15121       2680     354     14                                 0                               0                             2684 ...            0
1  15122       2683       0     13                                 0                               0                             2654 ...            0
2  15123       2713      16     15                                 0                               0                             2980 ...            0
3  15124       2709      24     17                                 0                               0                             2950 ...            0
4  15125       2706      29     19                                 0                               0                             2920 ...            0
[47 more columns]


In [139]:
print(train.columns)

Index(['Id', 'Elevation', 'Aspect', 'Slope',
       'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology',
       'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon',
       'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points',
       'Wilderness_Area1', 'Wilderness_Area2', 'Wilderness_Area3',
       'Wilderness_Area4', 'Soil_Type1', 'Soil_Type2', 'Soil_Type3',
       'Soil_Type4', 'Soil_Type5', 'Soil_Type6', 'Soil_Type7', 'Soil_Type8',
       'Soil_Type9', 'Soil_Type10', 'Soil_Type11', 'Soil_Type12',
       'Soil_Type13', 'Soil_Type14', 'Soil_Type15', 'Soil_Type16',
       'Soil_Type17', 'Soil_Type18', 'Soil_Type19', 'Soil_Type20',
       'Soil_Type21', 'Soil_Type22', 'Soil_Type23', 'Soil_Type24',
       'Soil_Type25', 'Soil_Type26', 'Soil_Type27', 'Soil_Type28',
       'Soil_Type29', 'Soil_Type30', 'Soil_Type31', 'Soil_Type32',
       'Soil_Type33', 'Soil_Type34', 'Soil_Type35', 'Soil_Type36',
       'Soil_Type37', 'Soil_Type38', 'Soil_Type39', 'Soil_

In [140]:
print(len(train.columns))

56


In [141]:
Id=test['Id']
print(train['Cover_Type'].head())

0    5
1    5
2    2
3    2
4    5
Name: Cover_Type, dtype: int64


In [142]:
def minus_one(num):
    return num - 1

y=(train['Cover_Type'].applymap(minus_one))

In [143]:

print(y)

0    4
1    4
2    1
3    1
4    4
5    1
6    4
7    4
8    4
9    4
[15110 more rows]
Name: Cover_Type, dtype: int64


In [144]:
print(Id.head())

0    15121
1    15122
2    15123
3    15124
4    15125
Name: Id, dtype: int64


In [145]:
print(y.head())

0    4
1    4
2    1
3    1
4    4
Name: Cover_Type, dtype: int64


In [146]:
print(train.describe())

   stats          Id    Elevation      Aspect      Slope  Horizontal_Distance_To_Hydrology  Vertical_Distance_To_Hydrology ...  Cover_Type
0  count     15120.0      15120.0     15120.0    15120.0                           15120.0                         15120.0 ...     15120.0
1   mean      7560.5  2749.322553  156.676653  16.501587                        227.195701                       51.076521 ...         4.0
2    std  4364.91237   417.678187  110.085801   8.453927                        210.075296                       61.239406 ...    2.000066
3    min         1.0       1863.0         0.0        0.0                               0.0                          -146.0 ...         1.0
4    25%     3780.75       2376.0        65.0       10.0                              67.0                             5.0 ...         2.0
5    50%      7560.5       2752.0       126.0       15.0                             180.0                            32.0 ...         4.0
6    75%    11340.25       

In [147]:
print(train['Cover_Type'].value_counts())
# q= cudf.DataFrame()
# q['Cover_Type']=train.loc[:,['Cover_Type']]
# q['Elevation']=train.loc[:,['Elevation']]


1    2160
2    2160
3    2160
4    2160
5    2160
6    2160
7    2160
dtype: int64


In [148]:
print(train.groupby(['Cover_Type']).sum())

         Id  Elevation  Aspect  Slope  Horizontal_Distance_To_Hydrology  Vertical_Distance_To_Hydrology  Horizontal_Distance_To_Roadways ...  Soil_Type40
Cover_Type
1  17271528    6756536  344441  28324                            586456                           89168                          5572186 ...           49
2  13635425    6312687  326370  28995                            621494                          102250                          5239710 ...            3
3  17555481    5180594  375133  44558                            455162                          138417                          2094327 ...            0
4  13725905    4802588  298295  39892                            225801                           86710                          1976617 ...            0
5  14011489    6019491  298063  36125                            451166                          109882                          2871328 ...            0
6  17412419    5234278  390133  41010                            

In [149]:
train = train.loc[:,['Elevation', 'Aspect', 'Slope','Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology',
       'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon','Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points',
       'Wilderness_Area1', 'Wilderness_Area2', 'Wilderness_Area3','Wilderness_Area4', 'Soil_Type1', 'Soil_Type2', 'Soil_Type3',
       'Soil_Type4', 'Soil_Type5', 'Soil_Type6', 'Soil_Type7', 'Soil_Type8','Soil_Type9', 'Soil_Type10', 'Soil_Type11', 'Soil_Type12',
       'Soil_Type13', 'Soil_Type14', 'Soil_Type15', 'Soil_Type16','Soil_Type17', 'Soil_Type18', 'Soil_Type19', 'Soil_Type20',
       'Soil_Type21', 'Soil_Type22', 'Soil_Type23', 'Soil_Type24','Soil_Type25', 'Soil_Type26', 'Soil_Type27', 'Soil_Type28',
       'Soil_Type29', 'Soil_Type30', 'Soil_Type31', 'Soil_Type32','Soil_Type33', 'Soil_Type34', 'Soil_Type35', 'Soil_Type36',
       'Soil_Type37', 'Soil_Type38', 'Soil_Type39', 'Soil_Type40']]

In [150]:
print(train.head())

   Elevation  Aspect  Slope  Horizontal_Distance_To_Hydrology  Vertical_Distance_To_Hydrology  Horizontal_Distance_To_Roadways  Hillshade_9am ...  Soil_Type40
0       2596      51      3                               258                               0                              510            221 ...            0
1       2590      56      2                               212                              -6                              390            220 ...            0
2       2804     139      9                               268                              65                             3180            234 ...            0
3       2785     155     18                               242                             118                             3090            238 ...            0
4       2595      45      2                               153                              -1                              391            220 ...            0
[46 more columns]


In [151]:
print(test.columns)

Index(['Id', 'Elevation', 'Aspect', 'Slope',
       'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology',
       'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon',
       'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points',
       'Wilderness_Area1', 'Wilderness_Area2', 'Wilderness_Area3',
       'Wilderness_Area4', 'Soil_Type1', 'Soil_Type2', 'Soil_Type3',
       'Soil_Type4', 'Soil_Type5', 'Soil_Type6', 'Soil_Type7', 'Soil_Type8',
       'Soil_Type9', 'Soil_Type10', 'Soil_Type11', 'Soil_Type12',
       'Soil_Type13', 'Soil_Type14', 'Soil_Type15', 'Soil_Type16',
       'Soil_Type17', 'Soil_Type18', 'Soil_Type19', 'Soil_Type20',
       'Soil_Type21', 'Soil_Type22', 'Soil_Type23', 'Soil_Type24',
       'Soil_Type25', 'Soil_Type26', 'Soil_Type27', 'Soil_Type28',
       'Soil_Type29', 'Soil_Type30', 'Soil_Type31', 'Soil_Type32',
       'Soil_Type33', 'Soil_Type34', 'Soil_Type35', 'Soil_Type36',
       'Soil_Type37', 'Soil_Type38', 'Soil_Type39', 'Soil_

In [152]:
test=test.loc[:,['Elevation', 'Aspect', 'Slope','Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology',
       'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon','Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points',
       'Wilderness_Area1', 'Wilderness_Area2', 'Wilderness_Area3','Wilderness_Area4', 'Soil_Type1', 'Soil_Type2', 'Soil_Type3',
       'Soil_Type4', 'Soil_Type5', 'Soil_Type6', 'Soil_Type7', 'Soil_Type8','Soil_Type9', 'Soil_Type10', 'Soil_Type11', 'Soil_Type12',
       'Soil_Type13', 'Soil_Type14', 'Soil_Type15', 'Soil_Type16','Soil_Type17', 'Soil_Type18', 'Soil_Type19', 'Soil_Type20',
       'Soil_Type21', 'Soil_Type22', 'Soil_Type23', 'Soil_Type24','Soil_Type25', 'Soil_Type26', 'Soil_Type27', 'Soil_Type28',
       'Soil_Type29', 'Soil_Type30', 'Soil_Type31', 'Soil_Type32','Soil_Type33', 'Soil_Type34', 'Soil_Type35', 'Soil_Type36',
       'Soil_Type37', 'Soil_Type38', 'Soil_Type39', 'Soil_Type40']]

In [153]:
print(test.head())

   Elevation  Aspect  Slope  Horizontal_Distance_To_Hydrology  Vertical_Distance_To_Hydrology  Horizontal_Distance_To_Roadways  Hillshade_9am ...  Soil_Type40
0       2680     354     14                                 0                               0                             2684            196 ...            0
1       2683       0     13                                 0                               0                             2654            201 ...            0
2       2713      16     15                                 0                               0                             2980            206 ...            0
3       2709      24     17                                 0                               0                             2950            208 ...            0
4       2706      29     19                                 0                               0                             2920            210 ...            0
[46 more columns]


In [154]:
from cuml.preprocessing.model_selection import train_test_split

In [155]:
x_train, x_test, y_train, y_test = train_test_split(train, y, train_size=0.7)

In [156]:
from cuml.ensemble import RandomForestClassifier as cuRFC
# from cuml.linear_model import LogisticRegression

In [157]:
cuml_model = cuRFC(n_estimators=300)
# reg = LogisticRegression()

In [159]:
# y_train = cudf.Series(np.asarray(y_train, dtype=np.int32))
y_train=(np.asarray(y_train, dtype=np.int32))
cuml_model.fit(x_train,y_train)
# reg.fit(x_train,y_train)

/conda/envs/rapids/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Using NumPy for number of class detection,install CuPy for faster processing.
  This is separate from the ipykernel package so we can avoid doing imports until


In [160]:
cuml_predict = cuml_model.predict(x_test)
# predict =reg.predict(x_test)

AttributeError: 'DataFrame' object has no attribute 'ctypes'

In [161]:
print(cuml_predict)

NameError: name 'cuml_predict' is not defined

array([5, 0, 3, ..., 2, 2, 5], dtype=int32)